MK Visualisasi Data Gasal 2020/2021

<b>Nama<b> : Irawansyah <br> 
<b>NIM<b>  : 1301174689

Import Library apa saja yang akan digunakan

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category20_16
output_notebook()
warnings.filterwarnings('ignore')

## 1 - Load Dataset

Pada implementasi kali ini, digunakan datasets market-index dengan tujuan untuk dapat memvisualisasi Data Interaktif Fluktuasi Index Saham. Jumlah row sebanyak  8964  dengan  9  atribut.

Import dataset yaitu market-index.csv

In [2]:
df = pd.read_csv("market-index.csv")

Menampilkan data teratas

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Day_Perc_Change,Stock
0,2009-01-20,8.223333,8.223333,7.883333,7.883333,6.351252,1200.0,0,IDX
1,2009-01-21,7.883333,7.883333,7.883333,7.883333,6.351252,0.0,0,IDX
2,2009-01-22,8.020000,8.020000,8.020000,8.020000,6.461360,300.0,1.70409944655615,IDX
3,2009-01-23,8.283334,8.283334,8.283334,8.283334,6.673516,1800.0,3.17907381955779,IDX
4,2009-01-26,8.233334,8.493333,8.233334,8.493333,6.842703,600.0,2.47251707402762,IDX


## 2 - Explorasi Data

Pada section ini, akan dilakukan eksplorasi pada dataset untuk mengidentifikasi hal-hal yang dianggap "Anomali" pada dataset seperti Mising Value, Inkonsistensi  Nilai, Duplicat Value

Melihat ukuran shape dari dataset

In [4]:
df.shape

(8964, 9)

Melihat Null Values pada Kolom

In [5]:
df.isnull().sum()

Date                0
Open               21
High               21
Low                21
Close              21
Adj Close          21
Volume             21
Day_Perc_Change     0
Stock               0
dtype: int64

Data Cleaning untuk menghapus row atau kolom yang memiliki missing values

In [6]:
df.dropna(inplace=True)
df.shape

(8943, 9)

Mengecek apakah masih memiliki missing values

In [7]:
df.isnull().sum().sum()

0

Melihat tipe data

In [8]:
df.dtypes

Date                object
Open               float64
High               float64
Low                float64
Close              float64
Adj Close          float64
Volume             float64
Day_Perc_Change     object
Stock               object
dtype: object

Mengubah Tipe data pada kolom Date menjadi Tanggal

In [9]:
df['Date'] = pd.to_datetime(df['Date'])
df.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,Day_Perc_Change,Stock
0,2009-01-20,8.223333,8.223333,7.883333,7.883333,6.351252,1200.0,0,IDX
1,2009-01-21,7.883333,7.883333,7.883333,7.883333,6.351252,0.0,0,IDX


Mengubah Tipe data pada kolom Day_Perc_Change menjadi float64

In [10]:
df['Day_Perc_Change'] = pd.to_numeric(df['Day_Perc_Change'],errors='coerce')
df.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,Day_Perc_Change,Stock
0,2009-01-20,8.223333,8.223333,7.883333,7.883333,6.351252,1200.0,0.0,IDX
1,2009-01-21,7.883333,7.883333,7.883333,7.883333,6.351252,0.0,0.0,IDX


Mendapatkan total durasi waktu dalam bentu hari

In [11]:
df["Date"].max() - df["Date"].min()

Timedelta('4380 days 00:00:00')

Melihat deskripi dari harga saham 

In [12]:
df.iloc[-90:].describe().astype(int)

,Open,High,Low,Close,Adj Close,Volume,Day_Perc_Change
count,90,90,90,90,90,90,90
mean,11723,11806,11618,11718,11718,4203704444,0
std,606,583,627,614,614,960838678,1
min,10551,10782,10519,10632,10632,2997810000,-5
25%,11232,11308,11100,11186,11186,3501960000,0
50%,11704,11785,11596,11683,11683,3878715000,0
75%,12132,12205,12025,12172,12172,4694565000,0
max,12965,12973,12857,12899,12899,7693430000,3


Mengubah data pada kolom Adj Close, Volume, Day Percentage Change menjadi 1 angka dibelakang koma

In [13]:
df.round(1)
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Day_Perc_Change,Stock
0,2009-01-20,8.223333,8.223333,7.883333,7.883333,6.351252,1200.0,0.000000,IDX
1,2009-01-21,7.883333,7.883333,7.883333,7.883333,6.351252,0.0,0.000000,IDX
2,2009-01-22,8.020000,8.020000,8.020000,8.020000,6.461360,300.0,1.704099,IDX
3,2009-01-23,8.283334,8.283334,8.283334,8.283334,6.673516,1800.0,3.179074,IDX
4,2009-01-26,8.233334,8.493333,8.233334,8.493333,6.842703,600.0,2.472517,IDX


In [14]:

volume = df[['Date', 'Volume', 'Stock']]


# 1 Adj Close

Visualisasi data interaktif untuk menampilkan nilai " Adj Close" atau Harga penutupan yang telah disesuaikan untuk setiap indeks pasar saham

In [15]:
adj_close = df[['Date', 'Adj Close', 'Stock']]

Membuat column data source pada parameter saham "Adj Close"

In [16]:
idx = adj_close.loc[adj_close['Stock'] == 'IDX']
nasdaq = adj_close.loc[adj_close['Stock'] == 'NASDAQ']
nikkei = adj_close.loc[adj_close['Stock'] == 'NIKKEI']

idx_data = ColumnDataSource(idx)
nasdaq_data = ColumnDataSource(nasdaq)
nikkei_data = ColumnDataSource(nikkei)

Membuat figure untuk memvisualisasikannya

In [17]:
adj_fig = figure(x_axis_type='datetime',
                  plot_height=500, plot_width=900,
                  title='Adj Close of Market Indeks',
                  toolbar_location='right')

In [18]:
adj_idx    = adj_fig.line('Date', 'Adj Close', source = idx_data, color = 'red', legend_label = 'IDX')
adj_nasdaq = adj_fig.line('Date', 'Adj Close', source = nasdaq_data, color = 'black', legend_label = 'Nasdaq')         
adj_nikkei = adj_fig.line('Date', 'Adj Close', source = nikkei_data, color = 'darkviolet', legend_label = 'Nikkei')

Membuat data yang ditampilkan pada hover 

In [19]:
hover_idx     = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Adj Close}'),], renderers=[adj_idx], mode='vline')
hover_nasdaq  = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Adj Close}'),], renderers=[adj_nasdaq], mode='vline')
hover_nikkei  = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Adj Close}'),], renderers=[adj_nikkei], mode='vline')

Membuat tampilan hover pada figure

In [20]:
adj_fig.add_tools(hover_idx)
adj_fig.add_tools(hover_nasdaq)
adj_fig.add_tools(hover_nikkei)

Mengatur posisi legend dan click policy yang berfungsi untuk dapat dihide

In [21]:
adj_fig.legend.location = 'top_right'
adj_fig.legend.click_policy = 'hide'
show(adj_fig)

# II. Volume

Visualisasi data interaktif untuk menampilkan nilai "Volume" atau Jumlah transaksi untuk setiap indeks pasar saham

In [22]:
volume = df[['Date', 'Volume', 'Stock']]

Membuat column data source pada parameter saham "Volume"

In [23]:
idx = volume.loc[volume['Stock'] == 'IDX']
nasdaq = volume.loc[volume['Stock'] == 'NASDAQ']
nikkei = volume.loc[volume['Stock'] == 'NIKKEI']

idx_data = ColumnDataSource(idx)
nasdaq_data = ColumnDataSource(nasdaq)
nikkei_data = ColumnDataSource(nikkei)

Membuat figure untuk memvisualisasikannya

In [24]:
volume_fig = figure(x_axis_type='datetime',
                  plot_height=500, plot_width=900,
                  title='Volume of Market Indeks',
                  toolbar_location='right')

In [25]:
volume_idx = volume_fig.line('Date', 'Volume', source = idx_data, color = 'red', legend_label = 'IDX')
volume_nasdaq = volume_fig.line('Date', 'Volume', source = nasdaq_data, color = 'black', legend_label = 'Nasdaq')         
volume_nikkei = volume_fig.line('Date', 'Volume', source = nikkei_data, color = 'darkviolet', legend_label = 'Nikkei')

Membuat data yang ditampilkan pada hover 

In [26]:
hover_idx     = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Volume}'),], renderers=[volume_idx], mode='vline')
hover_nasdaq  = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Volume}'),], renderers=[volume_nasdaq], mode='vline')
hover_nikkei  = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Volume}'),], renderers=[volume_nikkei], mode='vline')

Membuat tampilan hover pada figure

In [27]:
volume_fig.add_tools(hover_idx)
volume_fig.add_tools(hover_nasdaq)
volume_fig.add_tools(hover_nikkei)

Mengatur posisi legend dan click policy yang berfungsi untuk dapat dihide

In [28]:
volume_fig.legend.location = 'top_right'
volume_fig.legend.click_policy = 'hide'
show(volume_fig)

# III Day_Per_Change

Visualisasi data interaktif untuk menampilkan nilai "Day_Per_Change" yaitu persentase perubahan harga saham untuk setiap indeks pasar saham

In [29]:
day = df[['Date', 'Day_Perc_Change', 'Stock']]

Membuat column data source pada parameter saham "Day_Per_Change"

In [30]:
idx = day.loc[day['Stock'] == 'IDX']
nasdaq = day.loc[day['Stock'] == 'NASDAQ']
nikkei = day.loc[day['Stock'] == 'NIKKEI']

idx_data = ColumnDataSource(idx)
nasdaq_data = ColumnDataSource(nasdaq)
nikkei_data = ColumnDataSource(nikkei)

Membuat figure untuk memvisualisasikannya

In [31]:
day_fig = figure(x_axis_type='datetime',
                  plot_height=500, plot_width=900,
                  title='Day_Per_Change of Market Indeks',
                  toolbar_location='right')

In [32]:
day_idx    = day_fig.line('Date', 'Day_Perc_Change', source = idx_data, color = 'red', legend_label = 'IDX')
day_nasdaq = day_fig.line('Date', 'Day_Perc_Change', source = nasdaq_data, color = 'black', legend_label = 'Nasdaq')         
day_nikkei = day_fig.line('Date', 'Day_Perc_Change', source = nikkei_data, color = 'darkviolet', legend_label = 'Nikkei')

Membuat data yang ditampilkan pada hover 

In [33]:
hover_idx     = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Day_Perc_Change}'),], renderers=[day_idx], mode='vline')
hover_nasdaq  = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Day_Perc_Change}'),], renderers=[day_nasdaq], mode='vline')
hover_nikkei  = HoverTool(tooltips=[('Name of market index','@Stock'),('Value', '@{Day_Perc_Change}'),], renderers=[day_nikkei], mode='vline')

Membuat tampilan hover pada figure

In [34]:
day_fig.add_tools(hover_idx)
day_fig.add_tools(hover_nasdaq)
day_fig.add_tools(hover_nikkei)

Mengatur posisi legend dan click policy yang berfungsi untuk dapat dihide

In [35]:
day_fig.legend.location = 'top_right'
day_fig.legend.click_policy = 'hide'
show(day_fig)

Membuat panel

In [36]:
adj_panel = Panel(child = adj_fig, title='Adj Close')
volume_panel = Panel(child = volume_fig, title='Volume')
day_panel = Panel(child = day_fig, title='Day_Per_Change')

Menampilkan visualisasi panel pada tab

In [37]:
tab = Tabs(tabs = [adj_panel, volume_panel, day_panel])
show(tab)